In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.types import Date
from sqlalchemy.orm import Session
from sqlalchemy import func
from sqlalchemy.sql.expression import and_, extract
from datetime import datetime, timedelta
import pandas as pd

from pprint import pprint


In [2]:
Base = declarative_base()

In [3]:
class M(Base):
    __tablename__ = "measurement"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

class Stations(Base):
    __tablename__ = "station"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [4]:
database_path = "Resources/hawaii.sqlite"
engine =  create_engine(f"sqlite:///{database_path}")
conn = engine.connect()
session = Session(bind=engine)

In [5]:
#find max date
max_date = session.query(func.max(M.date)).first()[0]
pprint(max_date)

datetime.date(2017, 8, 23)


In [6]:
one_year_ago = max_date - timedelta(days=365)
one_year_ago


datetime.date(2016, 8, 23)

In [11]:
my_data = session.query(M).filter(M.date >= one_year_ago,  M.date <= max_date)

pprint(type(my_data))
pprint(my_data)

all_info_df = pd.read_sql(my_data.statement, my_data.session.bind)
all_info_df

<class 'sqlalchemy.orm.query.Query'>


,id,station,date,prcp,tobs
0,2364,USC00519397,2016-08-23,0.00,81.0
1,2365,USC00519397,2016-08-24,0.08,79.0
2,2366,USC00519397,2016-08-25,0.08,80.0
3,2367,USC00519397,2016-08-26,0.00,79.0
4,2368,USC00519397,2016-08-27,0.00,77.0
...,...,...,...,...,...
2225,19546,USC00516128,2017-08-19,0.09,71.0
2226,19547,USC00516128,2017-08-20,NaN,78.0
2227,19548,USC00516128,2017-08-21,0.56,76.0
2228,19549,USC00516128,2017-08-22,0.50,76.0


In [16]:
my_data = session.query(M.prcp, M.date).filter(M.date >= one_year_ago,  M.date <= max_date)

pprint(type(my_data))
pprint(my_data)

prcp_date_df = pd.read_sql(my_data.statement, my_data.session.bind)
prcp_date_df

index_prcp_date_df = pd.read_sql(my_data.statement, my_data.session.bind).set_index("date")
index_prcp_date_df

<class 'sqlalchemy.orm.query.Query'>


,prcp
date,
2016-08-23,0.00
2016-08-24,0.08
2016-08-25,0.08
2016-08-26,0.00
2016-08-27,0.00
...,...
2017-08-19,0.09
2017-08-20,NaN
2017-08-21,0.56


In [8]:
data = engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()
data

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]

In [9]:
for row in data:
    print()